subjectlist 처음뽑고, 뭔가 정리 하는거

In [1]:
import psycopg2
from dfply import *
import pandas as pd

### 1. DB 연결

#### 1.1. DB 관련 파라미터 설정

In [11]:
from sshtunnel import SSHTunnelForwarder
############ DB Parameter 설정 (필요에 따라 수정해서 사용) ############

ssh_ip='210.91.223.248'
ssh_port=35430
ssh_user="medicalai"
ssh_pw="medicalai731!"
#-----------------------------------------------------------------------
database_name = 'mimiciv'
user = 'mai_admin'
password = 'admin1q2w!@'
host = 'localhost'
port = 35432
######################################################################

# 삽관/발관 코드 (이 지정값들을 기준으로 'intubation_all', 'extubation_all' 데이터 추출)
int_itemids = "224385"
ext_itemids = "225468, 225477, 227194"

# 필요한 테이블 쿼리 '(저장될 dataframe 이름): (SQL 쿼리문)'. 필요에 따라 쿼리문을 추가, 삭제, 수정해서 사용
tables_query = {
    'pg_tables': "SELECT * FROM PG_TABLES;",  # DB에 포함된 전체 테이블 리스트
    'patients': "SELECT * FROM mimiciv_hosp.patients;",  # 환자 기록
    'admissions': "SELECT * FROM mimiciv_hosp.admissions;",  # 입원 기록
    'transfers': "SELECT * FROM mimiciv_hosp.transfers;",  # 병동 이동 기록
    'icustays': "SELECT * FROM mimiciv_icu.icustays;",  # 중환자실 기록
    'd_items': "SELECT * FROM mimiciv_icu.d_items;",  # 아이템 딕셔너리
    'intubation_all': f"SELECT * FROM mimiciv_icu.procedureevents WHERE itemid IN ({int_itemids});",  # 삽관 이벤트
    'extubation_all': f"SELECT * FROM mimiciv_icu.procedureevents WHERE itemid IN ({ext_itemids});"  # 발관 이벤트
}


# 파라미터 로그
print("--------- Database Configuration ---------")
print("Database Name:", database_name)
print("User:", user)
print("Password:", "*" * len(password)) 
print("Host:", host)
print("Port:", port)
print()

print("--------- Intubation and Extubation Codes ---------")
print("Intubation Item ID:", int_itemids)
print("Extubation Item IDs:", ext_itemids)
print()

print("--------- SQL Queries for Required Tables ---------")
for table_name, query in tables_query.items():
    print(f"{table_name}: {query}")


--------- Database Configuration ---------
Database Name: mimiciv
User: mai_admin
Password: ***********
Host: localhost
Port: 35432

--------- Intubation and Extubation Codes ---------
Intubation Item ID: 224385
Extubation Item IDs: 225468, 225477, 227194

--------- SQL Queries for Required Tables ---------
pg_tables: SELECT * FROM PG_TABLES;
patients: SELECT * FROM mimiciv_hosp.patients;
admissions: SELECT * FROM mimiciv_hosp.admissions;
transfers: SELECT * FROM mimiciv_hosp.transfers;
icustays: SELECT * FROM mimiciv_icu.icustays;
d_items: SELECT * FROM mimiciv_icu.d_items;
intubation_all: SELECT * FROM mimiciv_icu.procedureevents WHERE itemid IN (224385);
extubation_all: SELECT * FROM mimiciv_icu.procedureevents WHERE itemid IN (225468, 225477, 227194);


In [6]:
# ############ DB Parameter 설정 (필요에 따라 수정해서 사용) ############

# import psycopg2
# from sshtunnel import SSHTunnelForwarder


# try:
#     tunnel= SSHTunnelForwarder(
#          ('210.91.223.248', 35430),
#          #ssh_private_key="</path/to/private/ssh/key>",
#          ### in my case, I used a password instead of a private key
#          ssh_username="medicalai",
#          ssh_password="medicalai731!", 
#          remote_bind_address=('localhost', 35432))
         
#     tunnel.start()
#     print("tunnel connected")

#     params = {
#              'database': 'mimiciv',
#              'user': 'mai_admin',
#              'password': 'admin1q2w!@',
#              'host': 'localhost',
#              'port': tunnel.local_bind_port
#              }

#     conn = psycopg2.connect(**params)
#     conn.autocommit = True
#     curs = conn.cursor()
#     print("database connected")

# except Exception as e:
#     print("Connection Failed due to {e}")

# # Remember to close the SSH tunnel
# #tunnel.close()

tunnel connected
database connected


#### 1.2. DB 연결, 데이터 추출

In [1]:
import psycopg2
from sshtunnel import SSHTunnelForwarder

try:
    tunnel= SSHTunnelForwarder(
         (ssh_ip, ssh_port),
         #ssh_private_key="</path/to/private/ssh/key>",
         ### in my case, I used a password instead of a private key
         ssh_username=ssh_user,
         ssh_password=ssh_pw, 
         remote_bind_address=(host,port))    
    
    
    # 연결 파라미터 설정
    params = {
        'database': database_name,  
        'user': user,    
        'password': password,   
        'host': host,   
        'port': tunnel.local_bind_port     
    }

    # PostgreSQL 데이터베이스 연결
    conn = psycopg2.connect(**params)
    conn.autocommit = True  
    curs = conn.cursor()
    print(f'{database_name} DATABASE CONNECTED.')

    # 테이블 불러오기
    print('RETRIEVING DATA...')

    dataframes = {}  # 테이블 저장소

    for table_name, query in tables_query.items():
        curs.execute(query)
        columns_name = [desc[0] for desc in curs.description]
        dataframes[table_name] = pd.DataFrame(curs.fetchall(), columns=columns_name)
        print(f'RETRIEVED {table_name}: {dataframes[table_name].shape}')
    print(f'{len(dataframes)} DATAFRAMES SAVED AT dataframes DICTIONARY.' )


except Exception as e:
    print(f"Connection failed due to: {e}")
finally:
    # Close the cursor and connection to clean up
    if conn:
        curs.close()
        conn.close()
print('CONNECTION CLOSED.')

# Dataframe을 개별 변수로 저장
for key in dataframes:
    globals()[key] = dataframes[key]

Connection failed due to: name 'ssh_ip' is not defined


NameError: name 'conn' is not defined

### 2. 데이터 정제

#### 2.1. 환자 정보 필터링 
- 성인 환자 (anchor age >= 18) >>

    - 병원 입원(hadm_id) 정보가 있는 환자 >>
    
        -  중환자실 입원(stay_id) 정보가 있는 환자

In [4]:
# 18세 이상 환자 필터링
adults_pat = patients >> filter_by(X.anchor_age >= 18)

# admissions (입원기록) 테이블 결합
adults_hadm = adults_pat >> left_join(admissions, by = "subject_id") \
    >> select("subject_id", "gender", "anchor_age", "hadm_id", "admittime", "dischtime", "deathtime")

# 입원정보(hadm_id) 없는 행 삭제
adults_hadm2 = adults_hadm >> mask(adults_hadm.hadm_id.notnull())

# icu (응급병동) 테이블 결합
adults_icu = adults_hadm2 >> left_join(icustays, by = ("subject_id", "hadm_id"))

# icu 입원정보(stay_id) 없는 행 삭제
adults_icu2 = adults_icu >> mask(adults_icu.stay_id.notnull())

adults_icu2.shape  # (73181, 13)


(73181, 13)

ICU 입원 기록이 있는 환자 기술통계

In [5]:
num_pat = len(adults_icu2.subject_id.unique())
icustay_per_adult = adults_icu2.groupby(['subject_id'])['stay_id'].agg('count')
mean_icustay_count = icustay_per_adult.mean()
median_icustay_count = icustay_per_adult.median()
std_dev_icustay_count = icustay_per_adult.std()
min_icustay_count = icustay_per_adult.min()
max_icustay_count = icustay_per_adult.max()

print('<환자별 중환자실(ICU) 입원 횟수 기술통계>')
print(f'고유 환자수: {"{:,}".format(num_pat)}명')
print(f"평균 입원 횟수: {round(mean_icustay_count, 2)} 회")
print(f"표준편차: {round(std_dev_icustay_count, 2)} 회")
print(f"중간값: {median_icustay_count} 회")
print(f"최솟값: {min_icustay_count} 회")
print(f"최댓값: {max_icustay_count} 회")

<환자별 중환자실(ICU) 입원 횟수 기술통계>
고유 환자수: 50,920명
평균 입원 횟수: 1.44 회
표준편차: 1.15 회
중간값: 1.0 회
최솟값: 1 회
최댓값: 37 회


#### 2.2. 삽관/발관 (intubation/extubation) 정보 필터링 


In [6]:
# 필요한 칼럼만 조회
intubation1 = intubation_all >> select("subject_id", "hadm_id", "stay_id", "starttime", "itemid", "patientweight")
extubation1 = extubation_all >> select("subject_id", "hadm_id", "stay_id", "starttime", "itemid", "patientweight")

# 칼럼명 변경, 시간자료 datetime 변환
intubation1.rename(columns={'starttime':'intubationtime'}, inplace=True)
extubation1.rename(columns={'starttime':'extubationtime'}, inplace=True)

intubation1['intubationtime'] = pd.to_datetime(intubation1['intubationtime'])
extubation1['extubationtime'] = pd.to_datetime(extubation1['extubationtime'])

# extubation 코드 유형 정리: 225477, 225468 = unplanned
def label_extubation(row):
    if row['itemid'] == 225477:
        return 'Unplanned Extuabtion (non-patient initiated)'
    elif row['itemid'] == 225468:
        return 'Unplanned Extuabtion (patient-initiated)'
    else:
        return 'Planned Extubation'
    
extubation1['extubationcause'] = extubation1.apply(lambda row: label_extubation(row), axis=1)

print(f'intubation1 shape: {intubation1.shape}')   # (8488, 6)
print(f'extubation1 shape: {extubation1.shape}')   # (23122, 6)

intubation1 shape: (8488, 6)
extubation1 shape: (23122, 7)


#### 2.3. 삽관/발관 (intubation/extubation) 중복값 삭제
- 참고: 현재는 완전히 동일한 중복값만 제거하도록 되어 있음. (time_diff=0) 데이터 alignment 작업 후에 이벤트 시간 차를 기준으로 데이터를 걸러주는 작업을 또 하기 때문임.  

In [61]:
# (오리지널 코드와 결과 비교 필요)
# 대표적인 차이점: 
### 먼저 subject_id, hadm_id, intubation/extubation time 이 "모두" 일치하는 중복데이터를 제거하고 시작했음. 
### groupby 함수를 이용해서 [subject_id, hadm_id]로 묶어준 다음에 비교 코드를 수행했음.
### 결과적으로, 이 방법을 사용했을 때 오리지널 코드보다 데이터 손실이 더 적었음. 이전 코드가 불필요한 행을 삭제한 것인지, 해당 코드가 삭제되어야 할 중복행을 놓친 것인지 검증 필요.
### 결과 데이터를 엑셀로 추출해서 육안으로 검증을 했을 때는 오류가 보이진 않았음. HJ님이 검증을 도와주면 좋을 것 같음.  

def filter_close_events(df, time_col, group_cols, time_diff=0):
    """
    같은 그룹 내에서 이전 이벤트와 0시간 이내에 발생한 이벤트를 필터링합니다.

    :param df: 처리할 DataFrame
    :param time_col: 시간 데이터가 포함된 열의 이름
    :param group_cols: 그룹화할 열의 리스트
    :param time_diff: 중복으로 처리되는 기준 시간
    :return: 필터링된 DataFrame
    """
    # 중복 제거
    df_deduped = df.drop_duplicates(subset=group_cols + [time_col])

    # 행을 시간 순으로 정렬
    df_sorted = df_deduped.sort_values(by=group_cols + [time_col])

    # 각 그룹에 적용할 함수 정의
    def filter_rows(group):
        # 이벤트 간 시간 차이 계산
        group['time_diff'] = group[time_col].diff()

        # 시간 차이가 1시간 이하인 행 표시
        mask = group['time_diff'] <= pd.Timedelta(hours=time_diff)

        # 연속된 쌍의 두 번째 행을 표시하기 위해 마스크를 이동
        mask_shifted = mask.shift(-1, fill_value=False)

        # 제거 표시가 안된 행만 유지
        return group[~mask_shifted]

    # 각 그룹에 함수 적용
    return df_sorted.groupby(group_cols, group_keys=False).apply(filter_rows).drop(columns=['time_diff'])

# intubation1 및 extubation1 DataFrame에 함수 적용
intubation1 = filter_close_events(intubation1, 'intubationtime', ['subject_id', 'hadm_id'])
extubation1 = filter_close_events(extubation1, 'extubationtime', ['subject_id', 'hadm_id'])

# 결과 확인
print(f'필터링된 삽관 데이터 크기: {intubation1.shape}')
print(f'필터링된 발관 데이터 크기: {extubation1.shape}')


필터링된 삽관 데이터 크기: (8232, 6)
필터링된 발관 데이터 크기: (22653, 7)


In [8]:
intubation1[intubation1.subject_id == 10098215]

,subject_id,hadm_id,stay_id,intubationtime,itemid,patientweight
61,10098215,20652197,37703075,2118-09-02 07:45:00,224385,66.0
62,10098215,21701015,30598744,2118-01-10 13:09:00,224385,57.0
63,10098215,22394571,32301420,2118-06-18 01:48:00,224385,48.7
64,10098215,22394571,32301420,2118-06-21 08:28:00,224385,48.7
65,10098215,22394571,32817342,2118-05-09 19:02:00,224385,60.0
66,10098215,22394571,36018186,2118-06-03 20:50:00,224385,55.4
67,10098215,22394571,36018186,2118-06-07 01:40:00,224385,55.4
68,10098215,22394571,36018186,2118-06-03 20:00:00,224385,55.4
69,10098215,29752040,31800901,2117-10-26 10:53:00,224385,56.5


In [62]:
# 검증코드: 두개 이상의 삽관/발관 기록이 있는 환자들만 따로 필터링 하여 삽관/발관 이벤트 시간 비교

# Intubation
intubation_filtered_indexed = filtered_intubation.set_index(['subject_id', 'hadm_id'])
intubation_filtered_sorted = intubation_filtered_indexed.sort_index()
intubation_filtered_by_hadm_id = filtered_intubation.groupby('subject_id').filter(lambda x: x['hadm_id'].nunique() > 1)
intubation_filtered_by_hadm_id_indexed = intubation_filtered_by_hadm_id.set_index(['subject_id', 'hadm_id'])
intubation_filtered_by_hadm_id_sorted = intubation_filtered_by_hadm_id_indexed.sort_index()

# Extubation
extubation_filtered_indexed = filtered_extubation.set_index(['subject_id', 'hadm_id'])
extubation_filtered_sorted = extubation_filtered_indexed.sort_index()
extubation_filtered_by_hadm_id = filtered_extubation.groupby('subject_id').filter(lambda x: x['hadm_id'].nunique() > 1)
extubation_filtered_by_hadm_id_indexed = extubation_filtered_by_hadm_id.set_index(['subject_id', 'hadm_id'])
extubation_filtered_by_hadm_id_sorted = extubation_filtered_by_hadm_id_indexed.sort_index()



In [63]:
intubation_filtered_by_hadm_id_sorted

stay_id      intubationtime  itemid  patientweight
subject_id hadm_id                                                      
10004401   27939719  31202136 2144-04-11 05:03:00  224385          120.0
           29988601  32773003 2144-01-27 19:00:00  224385           76.0
           29988601  32773003 2144-01-30 13:00:00  224385           76.0
10098215   20652197  37703075 2118-09-02 07:45:00  224385           66.0
           21701015  30598744 2118-01-10 13:09:00  224385           57.0
...                       ...                 ...     ...            ...
19956723   23886623  33940420 2194-10-28 07:00:00  224385           92.0
           27397573  38452346 2194-11-21 17:15:00  224385           85.0
19960105   20796727  31681448 2146-08-19 04:35:00  224385           61.2
           27292691  31512322 2147-08-09 01:10:00  224385           56.8
           27292691  31512322 2147-08-15 12:40:00  224385           56.8

[842 rows x 4 columns]

In [64]:
# 검증 코드: intubation / extubation 테이블에서 중첩되는 ["subject_id", "hadm_id"] 그룹 숫자 구하기   # 5,601 그룹이 존재하는 것으로 나타남.

pat_int_group = filtered_intubation.groupby(["subject_id", "hadm_id"])[['intubationtime', 'itemid']].count()
pat_ext_group = filtered_extubation.groupby(["subject_id", "hadm_id"])[['extubationtime', 'itemid']].count()
unique_indices_int = pat_int_group.index.unique()
unique_indices_ext = pat_ext_group.index.unique()


unique_indices_int = pat_int_group.index.unique()
unique_indices_ext = pat_ext_group.index.unique()

common_indices = unique_indices_int.intersection(unique_indices_ext)

count_of_common_indices = len(common_indices)

print(f'Number of common "index" combinations between pat_int_group and pat_ext_group: {count_of_common_indices}')


Number of common "index" combinations between pat_int_group and pat_ext_group: 5601


#### 2.4. 삽관/발관 (intubation/extubation) 테이블 결합

In [9]:

intubation_extubation = intubation1 >> left_join(extubation1, by=("subject_id", "hadm_id"))
# intubation_extubation_inner = filtered_intubation >> inner_join(filtered_extubation, by=("subject_id", "hadm_id"))  # 테스트용. intubation/extubation 값이 모두 존재하는 데이터는 9387 건 있는 것으로 나옴.
# intubation_extubation_stay = intubation1 >> left_join(extubation1, by=("subject_id", "hadm_id", "stay_id"))  # 테스트용. 'stay_id' 기준으로 결합

# 칼럼명 정리
intubation_extubation.rename(columns={'stay_id_x': 'int_stayid' , 'itemid_x':'int_itemid', 'itemid_y':'ext_itemid', 'patientweight_x':'int_weight', 'stay_id_y': 'ext_stayid', 'patientweight_y':'ext_weight'}, inplace=True)
# intubation_extubation_stay.rename(columns={'itemid_x':'int_itemid', 'itemid_y':'ext_itemid', 'patientweight_x':'int_weight', 'patientweight_y':'ext_weight'}, inplace=True)


intubation_extubation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11528 entries, 0 to 11527
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   subject_id       11528 non-null  int64         
 1   hadm_id          11528 non-null  int64         
 2   int_stayid       11528 non-null  int64         
 3   intubationtime   11528 non-null  datetime64[ns]
 4   int_itemid       11528 non-null  int64         
 5   int_weight       11528 non-null  float64       
 6   ext_stayid       9941 non-null   float64       
 7   extubationtime   9941 non-null   datetime64[ns]
 8   ext_itemid       9941 non-null   float64       
 9   ext_weight       9941 non-null   float64       
 10  extubationcause  9941 non-null   object        
dtypes: datetime64[ns](2), float64(4), int64(4), object(1)
memory usage: 990.8+ KB


#### 2.5. 입원/퇴원시각, 사망시각, 삽관/발관 시간차 변수 추가

In [10]:
admissions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431231 entries, 0 to 431230
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   subject_id            431231 non-null  int64         
 1   hadm_id               431231 non-null  int64         
 2   admittime             431231 non-null  datetime64[ns]
 3   dischtime             431231 non-null  datetime64[ns]
 4   deathtime             8598 non-null    datetime64[ns]
 5   admission_type        431231 non-null  object        
 6   admit_provider_id     431227 non-null  object        
 7   admission_location    431231 non-null  object        
 8   discharge_location    312076 non-null  object        
 9   insurance             431231 non-null  object        
 10  language              431231 non-null  object        
 11  marital_status        421998 non-null  object        
 12  race                  431231 non-null  object        
 13 

In [11]:
# 발관시간 - 삽관시간 계산
## 참고: 이 시간차 데이터는 잠깐 EDA하기 위한 용도임. 실제 전처리 코드에서는 사용되지 않는 칼럼임.
intubation_extubation['int_ext_time'] = intubation_extubation['extubationtime'] - intubation_extubation['intubationtime']
# 입원시각, 퇴원시각, 사망시각 결합
intubation_extubation = intubation_extubation >> left_join(admissions, by = ("subject_id", "hadm_id")) >> select("subject_id", "hadm_id", "int_stayid", "admittime", "intubationtime","int_itemid","int_weight", "ext_stayid", "extubationtime","ext_itemid",'ext_weight',"extubationcause","int_ext_time","dischtime", "deathtime")

intubation_extubation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11528 entries, 0 to 11527
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   subject_id       11528 non-null  int64          
 1   hadm_id          11528 non-null  int64          
 2   int_stayid       11528 non-null  int64          
 3   admittime        11528 non-null  datetime64[ns] 
 4   intubationtime   11528 non-null  datetime64[ns] 
 5   int_itemid       11528 non-null  int64          
 6   int_weight       11528 non-null  float64        
 7   ext_stayid       9941 non-null   float64        
 8   extubationtime   9941 non-null   datetime64[ns] 
 9   ext_itemid       9941 non-null   float64        
 10  ext_weight       9941 non-null   float64        
 11  extubationcause  9941 non-null   object         
 12  int_ext_time     9941 non-null   timedelta64[ns]
 13  dischtime        11528 non-null  datetime64[ns] 
 14  deathtime        3105 

### 3. 데이터 저장

1. 성인 ICU 환자 데이터 ('adults_icu2.csv')
2. 삽관/발관 이벤트 데이터 ('intubation_extubation.csv')
3. (검증 데이터) 중복값 필터링 후 groupby 된 intubation 데이터 ('intubation_filtered_by_hadm_id_sorted.csv') 
4. (검증 데이터) 중복값 필터링 후 groupby 된 extubation 데이터 ('extubation_filtered_by_hadm_id_sorted.csv')

In [76]:
# 환자, 삽관/발관 데이터 저장
adults_icu2.to_csv('../outputs/adults_icu2.csv', index=False)
intubation_extubation.to_csv('../outputs/intubation_extubation_raw.csv', index=False)   # subject_id, hadm_id joined

# 검증용 데이터 저장
# intubation_filtered_by_hadm_id_sorted.to_csv('../outputs/intubation_filtered_by_hadm_id_sorted.csv')
# extubation_filtered_by_hadm_id_sorted.to_csv('../outputs/extubation_filtered_by_hadm_id_sorted.csv')